In [1]:
#!pip install "unstructured[all-docs]" pillow pydantic lxml matplotlib

In [2]:
#!pip install unstructured-pytesseract

In [3]:
from unstructured.partition.pdf import partition_pdf
import pytesseract
from PIL import Image

In [4]:
from unstructured.partition.pdf import partition_pdf
raw_pdf_elements=partition_pdf(
                                filename="201409_cfpb_report_proxy-methodology.pdf",
                                strategy="hi_res",
                                extract_images_in_pdf=True,
                                extract_image_block_types=["Image", "Table"],
                                extract_image_block_to_payload=False,
                                extract_image_block_output_dir="extracted_data"
                              )

In [5]:
#raw_pdf_elements

In [6]:
Header=[]
Footer=[]
Title=[]
NarrativeText=[]
Text=[]
ListItem=[]
img=[]
tab=[]


for element in raw_pdf_elements:
  if "unstructured.documents.elements.Header" in str(type(element)):
            Header.append(str(element))
  elif "unstructured.documents.elements.Footer" in str(type(element)):
            Footer.append(str(element))
  elif "unstructured.documents.elements.Title" in str(type(element)):
            Title.append(str(element))
  elif "unstructured.documents.elements.NarrativeText" in str(type(element)):
            NarrativeText.append(str(element))
  elif "unstructured.documents.elements.Text" in str(type(element)):
            Text.append(str(element))
  elif "unstructured.documents.elements.ListItem" in str(type(element)):
            ListItem.append(str(element))
  elif "unstructured.documents.elements.Image" in str(type(element)):
            img.append(str(element))
  elif "unstructured.documents.elements.Table" in str(type(element)):
            tab.append(str(element))


In [16]:
#ListItem

['1. Executive summary .............................................................................................. 3',
 '3. Using census geography and surname data to construct proxies',
 '4. Assessing the ability to predict race and ethnicity: an application',
 '5. Conclusion .......................................................................................................... 23',
 '6. Technical Appendix A: Constructing the BISG probability ............................. 24',
 '7. Technical Appendix B: Receiver Operating Characteristics and',
 'USING PUBLICLY AVAILABLE INFORMATION TO PROXY FOR UNIDENTIFIED RACE AND ETHNICITY',
 'USING PUBLICLY AVAILABLE INFORMATION TO PROXY FOR UNIDENTIFIED RACE AND ETHNICITY',
 '2 12 C.F.R. § 1002.5(a)(2) and 12 C.F.R. § 1002.13. For HMDA and its implementing regulation, Regulation C, see 29 U.S.C § 2801-2810 and 12 C.F.R. Part 1003. For the Regulation B provisions concerning requests for information generally, see 12 C.F.R. § 1002.5.',
 '3 Th

In [17]:
#!pip install langchain_core

In [18]:

#!pip install langchain_openai

In [19]:

len(img)

7

In [20]:
len(tab)

14

In [23]:

from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
     

In [24]:
prompt_text = """You are an assistant tasked with summarizing tables for retrieval. \
    These summaries will be embedded and used to retrieve the raw table elements. \
    Give a concise summary of the table that is well optimized for retrieval. Table {element} """

In [25]:

prompt = ChatPromptTemplate.from_template(prompt_text)

In [28]:
from langchain_openai import AzureOpenAIEmbeddings
from langchain_openai import AzureOpenAI
from langchain.chat_models import AzureChatOpenAI
from dotenv import load_dotenv
import os

In [43]:
load_dotenv() # Load environment variables from the .env file
deployment_name                       = "CART"
embedding_deployment_name             = os.getenv("embedding_deployment_name") 
AZURE_OPENAI_API_TYPE                 = os.getenv("AZURE_OPENAI_API_TYPE")
AZURE_OPENAI_API_KEY                  = os.getenv("AZURE_OPENAI_API_KEY")
AZURE_OPENAI_ENDPOINT                 = os.getenv("AZURE_OPENAI_API_BASE")
AZURE_OPENAI_API_VERSION              = os.getenv("AZURE_OPENAI_API_VERSION_CHAT")
AZURE_OPENAI_API_VERSION_EMBEDDING    = os.getenv("AZURE_OPENAI_API_VERSION_EMBEDDING")

os.environ["OPENAI_API_VERSION"]      = AZURE_OPENAI_API_VERSION
os.environ["AZURE_OPENAI_ENDPOINT"]   = AZURE_OPENAI_ENDPOINT
os.environ["AZURE_OPENAI_API_KEY"]    = AZURE_OPENAI_API_KEY

print("Establishing connection with GPT-4 Turbo OpenAI LLM.")
llm = AzureChatOpenAI(
                        deployment_name = deployment_name,
                        temperature=0.0
                    )
print("Established connection with GPT-4 Turbo OpenAI LLM.")


print("Establishing connection with gpt-4o-mini OpenAI LLM.")
llm_vision = AzureChatOpenAI(
                        deployment_name = vision_model,
                        temperature=0.0
                    )
print("Established connection with gpt-4o-mini OpenAI LLM.")

print("Fetching text-embedding-ada-002 OpenAI Embeddings.")
embeddings_model = AzureOpenAIEmbeddings(
                                            model          = "text-embedding-ada-002",
                                            azure_endpoint = AZURE_OPENAI_ENDPOINT,
                                            api_key        = AZURE_OPENAI_API_KEY,
                                            openai_api_version = AZURE_OPENAI_API_VERSION_EMBEDDING
                                        )
print("Fetched with text-embedding-ada-002 OpenAI Embeddings.")

Establishing connection with GPT-4 Turbo OpenAI LLM.
Established connection with GPT-4 Turbo OpenAI LLM.
Establishing connection with gpt-4o-mini OpenAI LLM.


NameError: name 'vision_model' is not defined

In [30]:
summarize_chain = {"element": lambda x: x} | prompt | llm | StrOutputParser()

In [31]:
summarize_chain

{
  element: RunnableLambda(lambda x: x)
}
| ChatPromptTemplate(input_variables=['element'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['element'], input_types={}, partial_variables={}, template='You are an assistant tasked with summarizing tables for retrieval.     These summaries will be embedded and used to retrieve the raw table elements.     Give a concise summary of the table that is well optimized for retrieval. Table {element} '), additional_kwargs={})])
| AzureChatOpenAI(client=<openai.resources.chat.completions.Completions object at 0x0000019D94D9C310>, async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x0000019D94D9E9E0>, temperature=0.0, model_kwargs={}, openai_api_key='420acd848cbd4f948a56bb6f839c1174', openai_proxy='', azure_endpoint='https://cfpb.openai.azure.com/', deployment_name='CART', openai_api_version='2023-03-15-preview', openai_api_type='azure')
| StrOutputParser()

In [32]:
table_summaries = []

In [34]:

table_summaries=summarize_chain.batch(tab,{"max_concurrency": 14})

In [35]:
table_summaries

['Table 3.1 outlines the data sources used, while Table 3.2 explains how the BISG probability was constructed. Table 4.1 shows the composition of lending by race and ethnicity, and Table 4.2 details the prediction of race and ethnicity for applicants.',
 'This table compares the number of records where the surname did or did not match in geocoded and non-geocoded data. There were 8 non-matching surnames in the non-geocoded data and 26,297 in the geocoded data. In contrast, there were 58 matching surnames in the non-geocoded data and 190,435 in the geocoded data.',
 'This table summarizes the percentage breakdown of reported ethnicity for Hispanic, White, Black, Asian/Pacific Islander, American Indian/Alaska Native, and Multiracial individuals across four different classifiers or proxies: Reported, BISG, Surname-only, and Geography-only.',
 "This table shows the proxy values for different ethnic groups based on the BISG, surname-only, and geography-only methods. The values range from 0.

In [36]:

import base64
import os
from langchain_core.messages import HumanMessage

In [37]:

def encode_image(image_path):
    """Getting the base64 string"""
    with open(image_path, "rb") as image_file:
        return base64.b64encode(image_file.read()).decode("utf-8")

In [38]:
def image_summarize(img_base64, prompt):
    """Make image summary"""


    chat = ChatOpenAI(model="gpt-4-vision-preview", max_tokens=1024)

    msg = chat.invoke(
        [
            HumanMessage(
                content=[
                    {"type": "text", "text": prompt},

                     {
                        "type": "image_url",
                        "image_url": {"url": f"data:image/jpeg;base64,{img_base64}"},
                    },
                ]
            )
        ]
    )
    return msg.content

In [39]:
def generate_img_summaries(path):
    """
    Generate summaries and base64 encoded strings for images
    path: Path to list of .jpg files extracted by Unstructured
    """

    # Store base64 encoded images
    img_base64_list = []

    # Store image summaries
    image_summaries = []

    # Prompt
    prompt = """You are an assistant tasked with summarizing images for retrieval. \
    These summaries will be embedded and used to retrieve the raw image. \
    Give a concise summary of the image that is well optimized for retrieval."""


    base64_image = encode_image(path)
    img_base64_list.append(base64_image)
    image_summaries.append(image_summarize(base64_image, prompt))

    return img_base64_list, image_summaries

In [40]:
fpath="extracted_data/figure-1-1.jpg"

In [41]:

img_base64_list,image_summaries=generate_img_summaries(fpath)

OpenAIError: The api_key client option must be set either by passing api_key to the client or by setting the OPENAI_API_KEY environment variable